# FaceMesh 人脸AU单元实时监测

In [1]:
# 导入基础库
import cv2
import mediapipe as mp
from tqdm import tqdm
import time

E:\Anaconda\Lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# 导入三维人脸关键点检测的模型
mp_face_mesh = mp.solutions.face_mesh
model = mp_face_mesh.FaceMesh(
    static_image_mode=False,
    refine_landmarks=True,
    max_num_faces=5, # 最多检测的人脸数量
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

In [3]:
# 导入可视化函数和可视化样式
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

In [4]:
# 处理单帧的函数
def process_frame(img):
    # 记录该祯开始处理的时间
    t0 = time.time()
    scaler = 1 # 文字大小
    
    # BGR to RGB
    img_RGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # 将RGB图像输入模型，并得到检测结果
    results = model.process(img_RGB)
    
    # 如果有检测到人脸，则开始绘制人脸曲面和重点区域轮廓线
    if results.multi_face_landmarks: # 如果有检测到人脸
        for face_landmarks in results.multi_face_landmarks: # 遍历每一张人脸
            # 绘制人脸轮廓线
            mp_drawing.draw_landmarks(
                image=img, # 输入图像
                landmark_list=face_landmarks, # 关键点
                connections=mp_face_mesh.FACEMESH_TESSELATION, # 连接点
                landmark_drawing_spec=mp_drawing.DrawingSpec(thickness=1,circle_radius=2,color=(66,77,229)), # 关键点样式,默认为None
                connection_drawing_spec=mp_drawing_styles.get_default_face_mesh_tesselation_style()
           )
        
            # 绘制人脸轮廓线、眼睫毛、眼珠、嘴巴
            mp_drawing.draw_landmarks(
                image=img, # 输入图像
                landmark_list=face_landmarks, # 关键点
                connections=mp_face_mesh.FACEMESH_CONTOURS, # 连接点
                landmark_drawing_spec=mp_drawing.DrawingSpec(thickness=1,circle_radius=5,color=(66,77,229)), # 关键点样式,默认为None
                connection_drawing_spec=mp_drawing_styles.get_default_face_mesh_contours_style()
           )
        
            # 绘制瞳孔区域
            mp_drawing.draw_landmarks(
                image=img, # 输入图像
                landmark_list=face_landmarks, # 关键点
                connections=mp_face_mesh.FACEMESH_IRISES, # 连接点
                landmark_drawing_spec=mp_drawing.DrawingSpec(thickness=1,circle_radius=1,color=(10,169,77)), # 关键点样式,默认为None
                connection_drawing_spec=mp_drawing_styles.get_default_face_mesh_iris_connections_style()
            )
    else:
        # 如果没有检测到人脸，则提示
        # No face detected的提示只能是英文，否则会显示?????
        img = cv2.putText(img, 'No face detected!', (25 * scaler, 50 * scaler), cv2.FONT_HERSHEY_SIMPLEX, 1.25 * scaler, (255, 0, 255), 2 * scaler)
        
    # 记录该帧处理完毕的时间
    t1 = time.time()
    
    # 计算每秒处理图像帧数FPS
    FPS = 1 / (t1 - t0)
    
    img = cv2.putText(img, 'FPS: '+ str(int(FPS)), (25 * scaler, 100 * scaler), cv2.FONT_HERSHEY_SIMPLEX, 1.25 * scaler, (255, 0, 255))
    
    return img

## 摄像头抽帧处理

In [5]:
# 调用摄像头获取每帧
# 导入opencv读取视频的库
import cv2
import time

# 获取摄像头，传入0表示获取系统默认摄像头。如果是MacOS，则需要传入1
cap = cv2.VideoCapture(0)

# 打开capture
cap.open(0)

# 无线循环，直至断开连接
while cap.isOpened():
    # 读取摄像头数据
    ret, frame = cap.read()
    if not ret:
        print("Ignoring empty")
        break
        
    # 处理帧
    frame = process_frame(frame)
    
    # 显示处理后的帧
    cv2.imshow('frame', frame)
    
    # 等待键盘输入，27为esc键，ord用于将字符转为ASCII码
    if cv2.waitKey(1) in [27, ord('q')]:
        break
        
# 释放摄像头
cap.release()

# 关闭所有窗口
cv2.destroyAllWindows()

E:\Anaconda\Lib\site-packages\google\protobuf\symbol_database.py:78: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


## 视频逐帧处理

In [24]:
def video_process(video_path = 'D:/Pythoncode/MediapipeModels/videos/test1.mp4'):
    figurehead = video_path.split('/')[-1]
    output_path = 'out-'+ figurehead
    
    print('video_path:', video_path)
    
    # 获取视频总帧数
    cap1 = cv2.VideoCapture(video_path)
    total_frames = 0
    while cap1.isOpened():
        ret, frame = cap1.read()
        total_frames += 1
        if not ret:
            break
    cap1.retrieve()
    print('total_frames:', total_frames)
    
    cap2 = cv2.VideoCapture(video_path)
    frame_step = (cap2.get(cv2.CAP_PROP_FRAME_WIDTH), cap2.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    fourcc = int(cap2.get(cv2.CAP_PROP_FOURCC))
    fps = cap2.get(cv2.CAP_PROP_FPS)
    
    out = cv2.VideoWriter(output_path, fourcc, fps,(int(frame_step[0]), int(frame_step[1])))
    
    # 进度条绑定视频的总帧数
    with tqdm(total=total_frames-1) as pbar:
        try:
            while cap2.isOpened():
                ret, frame = cap2.read()
                if not ret:
                    break
                    
                #处理帧
                try:
                    frame = process_frame(frame)
                except:
                    print('error')
                    pass
                
                if ret:
                    out.write(frame)
                    # 更新进度条
                    pbar.update(1)
                    
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break
                    
        except:
            print('Midway interruption')
            pass
        
    cv2.destroyAllWindows()
    out.release()
    cap2.release()
    print('Finished!', output_path)
                    

In [25]:
# 处理视频
video_process()

video_path: D:/Pythoncode/MediapipeModels/videos/test1.mp4
total_frames: 451


100%|██████████| 450/450 [00:12<00:00, 35.56it/s]

Finished! out-test1.mp4
